In [0]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

import warnings
warnings.filterwarnings("ignore")

from torch.utils.data import Dataset
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
import joblib

from tqdm import tqdm_notebook, tqdm

In [2]:
cd /content/drive/My Drive/kaggle/Bengali.AI competition

/content/drive/My Drive/kaggle/Bengali.AI competition


모든 dataset을 사용하지 않고 일부만 사용

In [3]:
if torch.cuda.is_available():
    device = 'cuda'
print(device)

cuda


# 1. Read dataset

In [0]:
train0 = pd.read_parquet('data/train_image_data_0.parquet')

In [0]:
train0.tail()

,image_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,32292,32293,32294,32295,32296,32297,32298,32299,32300,32301,32302,32303,32304,32305,32306,32307,32308,32309,32310,32311,32312,32313,32314,32315,32316,32317,32318,32319,32320,32321,32322,32323,32324,32325,32326,32327,32328,32329,32330,32331
50205,Train_50205,247,250,250,250,252,250,250,250,251,250,251,250,251,250,251,254,253,253,253,253,253,253,253,253,253,253,253,253,253,253,254,254,254,254,253,252,251,251,251,...,251,252,251,250,248,246,248,250,252,251,250,248,245,243,244,247,251,251,249,248,248,250,253,253,249,247,249,252,252,250,246,248,252,252,249,248,250,247,243,243
50206,Train_50206,250,251,250,244,245,243,246,248,246,247,248,251,249,247,250,251,249,248,249,247,249,251,248,245,248,250,248,248,245,246,247,244,244,247,248,248,244,236,240,...,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,253
50207,Train_50207,246,245,246,246,245,245,247,247,245,246,246,245,246,247,247,246,247,248,245,242,244,247,247,247,247,247,244,244,246,246,245,245,247,247,246,246,247,246,244,...,244,245,247,248,246,244,245,247,248,248,247,245,242,243,244,243,246,250,249,246,247,248,246,246,247,246,245,245,246,248,247,246,248,246,245,248,249,246,243,245
50208,Train_50208,246,242,240,244,247,242,239,241,245,245,243,244,245,243,244,247,247,245,241,240,244,246,244,240,241,245,245,241,243,245,244,242,243,243,241,241,243,245,245,...,250,250,249,246,244,244,243,242,242,243,244,243,241,241,243,247,250,250,247,247,250,248,248,249,248,247,248,251,251,249,247,247,247,247,247,246,246,248,248,247
50209,Train_50209,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255


colab에서 계속 Out of memory가 떠서 data의 양을 대폭 줄여서 train 한다.

In [5]:
df_train = pd.read_csv("data/train.csv")
df_train = df_train.loc[:5000]
df_train.tail()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
4996,Train_4996,13,7,4,ক্যে
4997,Train_4997,59,7,2,র্ণে
4998,Train_4998,103,1,5,ফ্রা
4999,Train_4999,112,7,0,ব্লে
5000,Train_5000,81,9,2,র্নো


In [0]:
print( df_train['grapheme_root'].nunique(), df_train['vowel_diacritic'].nunique(), df_train['consonant_diacritic'].nunique())

168 11 7


# 2. Split dataset

In [6]:
!pip install iterative-stratification

In [0]:
df_train['id'] = df_train['image_id'].apply(lambda x : int(x.split('_')[1]))

X = df_train.to_numpy()[:, 4] # id
y = df_train.to_numpy()[:, 1:4] # target values

In [0]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

mskf = MultilabelStratifiedKFold(n_splits=6, random_state=0)

In [0]:
df_train['fold'] = -1

for i, (trn_idx, vld_idx) in enumerate(mskf.split(X,y)):
    df_train.loc[vld_idx, 'fold'] = i

In [10]:
df_train['fold'].value_counts()

5    834
4    834
0    834
3    833
2    833
1    833
Name: fold, dtype: int64

In [0]:
trn_fold = [i for i in range(6) if i not in [5]]
vld_fold = [5]
trn_idx = df_train.loc[df_train['fold'].isin(trn_fold)].index
vld_idx = df_train.loc[df_train['fold'].isin(vld_fold)].index

In [0]:
trn_idx

Int64Index([   0,    1,    2,    4,    5,    6,    7,    8,    9,   10,
            ...
            4990, 4991, 4992, 4993, 4994, 4995, 4996, 4998, 4999, 5000],
           dtype='int64', length=4167)

# 3. Define dataset

In [0]:
class BengaliDataset(Dataset):
    def __init__(self, csv, img_height, img_width, transform):
        self.csv = csv.reset_index()
        self.img_idx = csv['image_id'].to_numpy()
        self.img_height = img_height
        self.img_width = img_width
        self.transform = transform

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):
        img_id = self.img_idx[idx]
        img = joblib.load(f'data/train_image/{img_id}.pkl')
        img = img.reshape(self.img_height, self.img_width).astype(np.int8)
        img = 255 - img
        # 3차원으로 만들기
        img = img[:,:,np.newaxis] 
        img = np.repeat(img, 3, 2)

        if self.transform is not None:
            img = self.transform(image=img)['image']

        label_1 = self.csv.iloc[idx].grapheme_root
        label_2 = self.csv.iloc[idx].vowel_diacritic
        label_3 = self.csv.iloc[idx].consonant_diacritic

        return img, np.array([label_1, label_2, label_3])

# 4. Define augmentations

In [13]:
!pip install albumentations

     |████████████████████████████████| 634kB 2.8MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=442a2eca65bfca6b055fc26bc7af2a70b7a86fb6c28680c073727f9d12d598ab
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [0]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose, Rotate
)

from albumentations.pytorch import ToTensor

In [0]:
train_augmentation = Compose([
                                Rotate(20),
                                ToTensor()
])

valid_augmentation = Compose([
                              ToTensor()
])

# 5. DataLoader

In [0]:
from torch.utils.data import DataLoader

In [0]:
trn_dataset = BengaliDataset(csv=df_train.loc[trn_idx],
                             img_height=137,
                             img_width=236,
                             transform=train_augmentation)

vld_dataset = BengaliDataset(csv=df_train.loc[vld_idx],
                             img_height=137,
                             img_width=236,
                             transform=train_augmentation)

In [0]:
# trn_dataset[0][0].shape # depth, width, height,

In [0]:
trn_loader = DataLoader(dataset=trn_dataset,
                        shuffle=True,
                        num_workers=2,
                        batch_size=16)

vld_loader = DataLoader(dataset=vld_dataset,
                        num_workers=2,
                        batch_size=16)

# 6. Model

In [31]:
!pip install pretrainedmodels

In [0]:
import pretrainedmodels

In [0]:
model_name = 'resnet34'
model = pretrainedmodels.__dict__[model_name](pretrained='imagenet')

In [0]:
in_features = model.last_linear.in_features
model.last_linear = torch.nn.Linear(in_features, 186) # 마지막 layer에서 out_features의 수를 바꾼다.

In [0]:
model = model.to(device) # gpu에 model 보내기 
# model = model.cuda() 이렇게 해도 된다고 한다

In [0]:
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
criterion = nn.CrossEntropyLoss()
schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       mode='max',
                                                       verbose=True,
                                                       patience=7,
                                                       factor=0.5)

# 7. train

In [0]:
train_loss = []

In [0]:
# 1 epoch만 진행한다 (37분 소요)
for inputs, targets in tqdm_notebook(trn_loader):
    inputs = inputs.to(device)
    targets = targets.to(device)

    outputs = model(inputs)

    grapheme = outputs[:, :168]
    vowel = outputs[:, 168:179]
    cons = outputs[:, 179:]

    loss = 0.5*criterion(grapheme, targets[:,0]) + 0.25*criterion(vowel, targets[:,1]) + 0.25*criterion(cons, targets[:,2])

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    train_loss.append(loss.item())

In [0]:
# 일단 model 저장
state_dict = model.to('cpu').state_dict() # state_dict 는 model의 parameter를 의미
model = model.to(device)
torch.save(state_dict, 'model.pt')

# model 불러오기
# model.load_state_dict(torch.load('model.pt'))

# 8. validation

In [38]:
model.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [52]:
val_loss = list()
val_true = list()
val_pred = list()
best_score = 0

model.eval() # dropout과 같은 val과정에서 하지 않는 연산끄기
with torch.no_grad(): # grad 연산끄기
    for inputs, targets in tqdm_notebook(vld_loader):
        inputs = inputs.cuda()
        targets = targets.cuda()

        outputs = model(inputs)

        grapheme = outputs[:, :168]
        vowel = outputs[:, 168:179]
        cons = outputs[:, 179:]

        loss = 0.5*criterion(grapheme, targets[:,0]) + 0.25*criterion(vowel, targets[:,1]) + 0.25*criterion(cons, targets[:,2])

        val_loss.append(loss.item())

        grapheme = grapheme.to('cpu').argmax(axis=1).data.numpy()
        vowel = vowel.to('cpu').argmax(axis=1).data.numpy()
        cons = cons.to('cpu').argmax(axis=1).data.numpy()

        val_true.append(targets.to('cpu').numpy())
        val_pred.append(np.stack([grapheme,vowel,cons], axis=1))

        val_true_concat = np.concatenate(val_true)
        val_pred_concat = np.concatenate(val_pred)
        # print(val_true_concat.shape, val_pred_concat.shape)

        val_loss_mean = np.mean(val_loss)
        train_loss_mean = np.mean(train_loss)
        # print(val_loss_mean, train_loss_mean)

        score_g = recall_score(val_true_concat[:,0], val_pred_concat[:,0], average='macro')
        score_v = recall_score(val_true_concat[:,1], val_pred_concat[:,1], average='macro')
        score_c = recall_score(val_true_concat[:,2], val_pred_concat[:,2], average='macro')
        # print(score_g, score_v, score_c)

        final_score = np.average([score_g, score_v, score_c], weights=[2,1,1])
        # print(f'train_loss : {train_loss:.5f}, val_loss : {val_loss.:5f}, final_socre : {final_score.:5f}')

In [56]:
print(val_true_concat.shape, val_pred_concat.shape)
print(score_g, score_v, score_c)
print(f'train_loss : {train_loss_mean:.5f}, val_loss : {val_loss_mean:.5f}, final_socre : {final_score:.5f}')

(834, 3) (834, 3)
0.009217510677242222 0.3005816217318093 0.15296726021850013
train_loss : nan, val_loss : 3.21785, final_socre : 0.11800
